In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow_probability as tfp

data = load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

input_dim = X_train.shape[1]
encoding_dim = 32

input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(encoding_dim, activation="relu")(input_layer)
decoder = tf.keras.layers.Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer="adam", loss="mse")

autoencoder.fit(X_train, X_train, epochs=100, batch_size=32, shuffle=True, validation_split=0.1, verbose=0)


def build_bayesian_nn(input_dim, output_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tfp.layers.DenseFlipout(64, activation="relu"),
        tfp.layers.DenseFlipout(64, activation="relu"),
        tfp.layers.DenseFlipout(output_dim, activation="sigmoid")
    ])

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

bayesian_nn = build_bayesian_nn(input_dim, 1)
bayesian_nn.fit(X_train, y_train, epochs=100, batch_size=32, shuffle=True, validation_split=0.1, verbose=0)


# Remove out-of-distribution samples using autoencoder
X_test_encoded = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - X_test_encoded, 2), axis=1)
threshold = np.percentile(mse, 95)  # Choose an appropriate threshold (e.g., 95th percentile)
inliers_idx = np.where(mse < threshold)[0]

# Evaluate Bayesian NN on the inliers
X_test_inliers = X_test[inliers_idx]
y_test_inliers = y_test[inliers_idx]

_, accuracy = bayesian_nn.evaluate(X_test_inliers, y_test_inliers, verbose=0)
print("Test0 accuracy:", accuracy)


def build_bayesian_nn(input_dim, output_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tfp.layers.DenseFlipout(64, activation="relu"),
        tfp.layers.DenseFlipout(64, activation="relu"),
        tfp.layers.DenseFlipout(output_dim)
    ])

    model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

bayesian_nn = build_bayesian_nn(input_dim, 2)
bayesian_nn.fit(X_train, y_train, epochs=100, batch_size=32, shuffle=True, validation_split=0.1, verbose=0)

n_forward_passes = 100

n_forward_passes = 100

def predict_with_uncertainty(model, X, n_forward_passes):
    preds = np.stack([model(X, training=True) for _ in range(n_forward_passes)])
    softmax_preds = tf.nn.softmax(preds, axis=-1)
    entropy = -np.sum(softmax_preds * np.log(softmax_preds + 1e-6), axis=-1)
    mean_entropy = np.mean(entropy, axis=0)
    return softmax_preds, mean_entropy

softmax_preds, mean_entropy = predict_with_uncertainty(bayesian_nn, X_test_inliers, n_forward_passes)

# Choose the class with the highest probability
y_pred = np.argmax(np.mean(softmax_preds, axis=0), axis=-1)

accuracy = np.mean(y_pred == y_test_inliers)
print("Test accuracy:", accuracy)



c:\Users\nstep\TSU\Samad\Uncertainty\.conda\lib\site-packages\tensorflow_probability\python\layers\util.py:102: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  trainable=trainable)
c:\Users\nstep\TSU\Samad\Uncertainty\.conda\lib\site-packages\tensorflow_probability\python\layers\util.py:112: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  trainable=trainable)


4/4 [==============================] - 0s 2ms/step
Test0 accuracy: 0.6851851940155029
Test accuracy: 0.6481481481481481
